In [0]:
from pyspark.sql.functions import to_date, col, round, current_timestamp
import sys
from time import time
from pyspark.sql.utils import AnalysisException

In [0]:
start_time = time()

In [0]:
expected_columns = {"customer_id", "customer_name", "country"}

try:
    df_bronze = spark.read.table("fz_catalog.bronze.customers")

    missing_columns = expected_columns - set(df_bronze.columns)
    if missing_columns:
        raise ValueError(
            f"Bronze table schema violation. "
            f"Missing column(s): {sorted(missing_columns)}"
        )

    df = df_bronze.select(*expected_columns)

except AnalysisException as e:
    # Table not found or Spark-level read issue
    raise AnalysisException(
        f"Failed to read Bronze table fz_catalog.bronze.customers: {e}"
    )

except ValueError as e:
    # Schema contract violation
    raise

In [0]:
df_silver = df.withColumn("created_date", current_timestamp())

In [0]:
df_silver.write.mode("overwrite").saveAsTable("fz_catalog.silver.customers")

In [0]:
end_time = time()

In [0]:
dbutils.notebook.exit(f"Run Time: {end_time - start_time} seconds")